In [1]:
from snowflake.snowpark import Session
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

import pandas as pd

In [2]:
df =  pd.read_csv('/data/Lumin_Dump_CreditRiskAnalytics_v1.csv')

In [3]:
df.shape

(356255, 108)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 108 entries, RECORD_COUNT to Credit Bureau Requests
dtypes: float64(71), int64(17), object(20)
memory usage: 293.5+ MB


In [6]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [7]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [8]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [ ]:
df_snowflake=session1.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_snowflake.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION")